<a href="https://colab.research.google.com/github/mehdihosseinimoghadam/Pytorch-Tutorial/blob/main/First.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

In [8]:

class DoubleConv(nn.Module):
  def __init__(self, in_channel, out_channel):
    super(DoubleConv, self).__init__()
    self.in_channel = in_channel
    self.out_channel = out_channel

    self.DC1 = nn.Sequential( 
        nn.Conv2d(self.in_channel,
                  self.out_channel,
                  3,
                  1,
                  1,
                  bias=False),
        nn.BatchNorm2d(self.out_channel),
        nn.ReLU()
        )      
        
    self.DC2 = nn.Sequential( 
        nn.Conv2d(self.in_channel,
                  self.out_channel,
                  5,
                  1,
                  2,
                  bias=False),
        nn.BatchNorm2d(self.out_channel),
        nn.ReLU()
        )  


    self.DC3 = nn.Sequential( 
        nn.Conv2d(self.in_channel,
                  self.out_channel,
                  7,
                  1,
                  3,
                  bias=False),
        nn.BatchNorm2d(self.out_channel),
        nn.ReLU()
        )  
  def forward(self, x):
    return(self.DC1(x),self.DC2(x), self.DC3(x))  

In [46]:
a = torch.rand(1,28,38).unsqueeze(0)

In [47]:
DC = DoubleConv(1,10)

In [48]:
x1, x2, x3 = DC(a)
print(x1.shape, x2.shape, x3.shape)
x = torch.cat([x1, x2, x3], axis = 1)
x = x.reshape((1, -1, x.shape[2]*x.shape[3]))
print(x.shape)


torch.Size([1, 10, 28, 38]) torch.Size([1, 10, 28, 38]) torch.Size([1, 10, 28, 38])
torch.Size([1, 30, 1064])


In [4]:
class RNN(nn.Module):
  def __init__(self, in_size, hidden_size, num_layers ):
    super(RNN, self).__init__()
    self.in_size = in_size
    self.num_layers = num_layers
    self.hidden_size = hidden_size

    self.rnn = nn.GRU(self.in_size, self.hidden_size, self.num_layers, batch_first=True,)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    x, _ = self.rnn(x, h0)
    return x  

In [49]:
a.shape

torch.Size([1, 1, 28, 38])

In [52]:
rnn = RNN(38, 50, 2)

In [53]:
rnn(a.squeeze(1)).shape

torch.Size([1, 28, 50])

In [127]:
rnn = nn.GRU(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)
print(output.shape)

torch.Size([5, 3, 20])


## My firs Arctechture

In [2]:
# Imports
import torch
import torchvision  # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For a nice progress bar!

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



class First(nn.Module):
  def __init__(self, in_channel, out_channel, hidden_size, num_layers, num_classes):
    super(First, self).__init__()
    self.in_channel = in_channel
    self.out_channel = out_channel
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.num_classes = num_classes

    self.DC1 = nn.Sequential( 
        nn.Conv2d(self.in_channel,
                  self.out_channel,
                  3,
                  1,
                  1,
                  bias=False),
        nn.BatchNorm2d(self.out_channel),
        nn.ReLU()
        )      
        
    self.DC2 = nn.Sequential( 
        nn.Conv2d(self.in_channel,
                  self.out_channel,
                  5,
                  1,
                  2,
                  bias=False),
        nn.BatchNorm2d(self.out_channel),
        nn.ReLU()
        )  


    self.DC3 = nn.Sequential( 
        nn.Conv2d(self.in_channel,
                  self.out_channel,
                  7,
                  1,
                  3,
                  bias=False),
        nn.BatchNorm2d(self.out_channel),
        nn.ReLU()
        )  
    self.rnn = nn.GRU(3*self.out_channel, self.hidden_size, self.num_layers, batch_first=True)

    self.fc1 = nn.Linear(28*28*50,256)
    self.fc2 = nn.Linear(256,self.num_classes)


  def forward(self, x):
    x = torch.cat([self.DC1(x),self.DC2(x), self.DC3(x)], axis = 1)
    # print(x.shape)
    x = x.reshape((x.shape[0], x.shape[2]*x.shape[3], -1))
    # print(x.shape)
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    # print(h0.shape)
    x, _ = self.rnn(x, h0)

    x = x.reshape(x.shape[0], -1) #####. Batch_size x multiplication of others
    x = self.fc1(x)
    x = self.fc2(x)

    return(x)

In [13]:
f = First(1,10,50,10,10)

In [14]:
b = torch.rand(64,1,28,28)

In [15]:
b.shape

torch.Size([64, 1, 28, 28])

In [16]:
print(f(b).shape)

torch.Size([64, 30, 28, 28])
torch.Size([64, 784, 30])
torch.Size([10, 64, 50])
torch.Size([64, 10])


In [3]:
# Hyperparameters
input_size = 28
hidden_size = 256
num_layers = 2
num_classes = 10
sequence_length = 28
learning_rate = 0.005
batch_size = 256
num_epochs = 3
# self, in_channel, out_channel, hidden_size, num_layers, num_classes)

In [ ]:
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [23]:
f = First(1,10,50,10,10)

In [5]:
model = First(1,10,50,10,10).to(device)
# Loss and optimizer




# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        # print(data.shape)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

torch.Size([256, 1, 28, 28])


 88%|████████▊ | 207/235 [01:49<00:14,  1.89it/s]

torch.Size([256, 1, 28, 28])


 89%|████████▊ | 208/235 [01:50<00:14,  1.89it/s]

torch.Size([256, 1, 28, 28])


...

torch.Size([256, 1, 28, 28])


100%|█████████▉| 234/235 [02:03<00:00,  1.89it/s]

torch.Size([96, 1, 28, 28])


100%|██████████| 235/235 [02:04<00:00,  1.89it/s]


Accuracy on training set: 97.55
Accuracy on test set: 97.62
